In [2]:
import numpy as np
import math
import csv
from statistics import mode
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Activation, Flatten, MaxPooling2D
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


In [3]:
# SET50 (unrealize)
SET50 = ['ADVANC', 'AOT', 'BANPU', 'BBL', 'BCP', 'BDMS',
        'BEM', 'BH', 'BJC', 'BTS', 'CENTEL', 'CPALL', 
         'CPF', 'CPN', 'DTAC', 'EGCO', 'GLOBAL', 'HMPRO', 
         'INTUCH', 'IRPC', 'KBANK', 'KCE', 'KKP', 'KTB', 'LH', 'MINT', 'PTT',
        'PTTEP', 'ROBINS', 'SCB', 'SCC', 
        'TCAP', 'TISCO', 'TMB', 'TOP', 'TRUE', 'TU']

# remove (low data) : TPIPP WHA SPRC SAWAD PSH MTLS IVL GPSC EA CBG BPP BEAUTY PTTGC

In [4]:
def load_stock_data(symbol, year_start, year_end):
    stock_data = []
    with open('./SET50/' + symbol + '.BK.csv', 'r') as csv_file:
        file_data = csv.reader(csv_file, delimiter=',')
        file_data = list(file_data)[1:]
        temp_value = 0
        for row in file_data:
            if row[2] is '':
                continue
            elif int(row[0][0:4]) >= year_start and int(row[0][0:4]) <= year_end:
                temp = (float(row[2]) + float(row[3]))/2.0
                if temp_value != 0:
                    unrealize = (temp - temp_value) / temp_value
                else:
                    unrealize = 0
                unrealize = ["{0:.4f}".format(unrealize)]
                stock_data.append(unrealize)
                temp_value = temp
                
    return stock_data

def mean_square_error(actual, predict):
    length = len(actual)
    sums_error = 0.0
    for i, j in zip(actual, predict):
        sums_error += (i - j)**2
    
    return sums_error / length

In [5]:
def findNewXTest(x_test, predict):
    for (index, element) in enumerate(x_test[0]):
        for i in range(len(element)):
            if i == 0:
                continue
            elif i < 29:
                element[i - 1] = element[i]
            else:
                element[i - 1] = element[i]
                element[i] = predict[index]
    return x_test

In [6]:
stock_datas = [ load_stock_data(x, 2007, 2017) for x in SET50 ]

min_count = len(stock_datas[0])

for index, element in enumerate(stock_datas):
    if len(element) < min_count:
        min_count = len(element)

min_count

2043

In [31]:
def create_model():
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), input_shape=(len(stock_datas), 30, 1), padding='valid'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))

    model.add(Conv2D(64, (3 ,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))   

    model.add(Flatten())  
    model.add(Dense(128))  
    model.add(Activation('relu'))  
    model.add(Dropout(0.5))
    model.add(Dense(len(stock_datas)))
    model.add(Activation('softmax'))
    
    return model

In [32]:
model_sgd = create_model()

optimizer = SGD(lr = 0.01, decay= 1e-6, momentum = 0.9, nesterov = True)

model_sgd.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

In [33]:
model_adam = create_model()

optimizer = Adam(lr = 0.01, decay= 1e-6)

model_adam.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

In [10]:
rounds = min_count - 30
train_index = int(rounds * 0.8)
validate_index = train_index + int(rounds * 0.1)

In [20]:
data = []
count = 0
index = 0
while index < rounds:
    temp = [i[index : index + 30] for i in stock_datas]
    data.append(temp)
    index += 1
    
x = np.asarray(data)
x_train = x[0: train_index].astype('float32')
x_validate = x[train_index: validate_index].astype('float32')
x_test = x[validate_index: validate_index + 1].astype('float32')

print(x_train.shape, x_validate.shape, x_test.shape)

(1610, 37, 30, 1) (201, 37, 30, 1) (1, 37, 30, 1)


In [12]:
target = []
index = 0
while index < rounds:
    temp = [stock_datas[i][index + 30][0] for i in range(len(stock_datas))]
    target.append(temp)
    index += 1
    
y = np.asarray(target)
y_train = y[0: train_index].astype('float32')
y_validate = y[train_index: validate_index].astype('float32')
y_test = y[validate_index: rounds].astype('float32')

print(y_train.shape, y_validate.shape, y_test.shape)

(1610, 37) (201, 37) (202, 37)


In [34]:
epochs = 100

model_sgd.fit(x_train, y_train, validation_data = (x_validate, y_validate), epochs = epochs, shuffle = True, batch_size = 100, verbose= 1)

Train on 1610 samples, validate on 201 samples
Epoch 1/100
1610/1610 [==============================] - 2s 968us/step - loss: 0.0016 - acc: 0.0224 - val_loss: 9.2411e-04 - val_acc: 0.0299
Epoch 2/100
1610/1610 [==============================] - 1s 462us/step - loss: 0.0016 - acc: 0.0286 - val_loss: 9.2411e-04 - val_acc: 0.0299
Epoch 3/100
1610/1610 [==============================] - 1s 465us/step - loss: 0.0016 - acc: 0.0292 - val_loss: 9.2411e-04 - val_acc: 0.0299
Epoch 4/100
1610/1610 [==============================] - 1s 462us/step - loss: 0.0016 - acc: 0.0224 - val_loss: 9.2411e-04 - val_acc: 0.0299
Epoch 5/100
1610/1610 [==============================] - 1s 465us/step - loss: 0.0016 - acc: 0.0230 - val_loss: 9.2411e-04 - val_acc: 0.0299
Epoch 6/100
1610/1610 [==============================] - 1s 471us/step - loss: 0.0016 - acc: 0.0273 - val_loss: 9.2411e-04 - val_acc: 0.0299
Epoch 7/100
1610/1610 [==============================] - 1s 465us/step - loss: 0.0016 - acc: 0.0230 - val_l

1610/1610 [==============================] - 0s 306us/step - loss: 0.0016 - acc: 0.0186 - val_loss: 9.2411e-04 - val_acc: 0.0348
Epoch 59/100
1610/1610 [==============================] - 0s 305us/step - loss: 0.0016 - acc: 0.0311 - val_loss: 9.2411e-04 - val_acc: 0.0348
Epoch 60/100
1610/1610 [==============================] - 1s 325us/step - loss: 0.0016 - acc: 0.0248 - val_loss: 9.2411e-04 - val_acc: 0.0348
Epoch 61/100
1610/1610 [==============================] - 1s 361us/step - loss: 0.0016 - acc: 0.0230 - val_loss: 9.2411e-04 - val_acc: 0.0348
Epoch 62/100
1610/1610 [==============================] - 1s 311us/step - loss: 0.0016 - acc: 0.0410 - val_loss: 9.2411e-04 - val_acc: 0.0348
Epoch 63/100
1610/1610 [==============================] - 0s 301us/step - loss: 0.0016 - acc: 0.0286 - val_loss: 9.2411e-04 - val_acc: 0.0348
Epoch 64/100
1610/1610 [==============================] - 0s 303us/step - loss: 0.0016 - acc: 0.0280 - val_loss: 9.2411e-04 - val_acc: 0.0348
Epoch 65/100
1610/1

In [35]:
model_adam.fit(x_train, y_train, validation_data = (x_validate, y_validate), epochs = epochs, shuffle = True, batch_size = 100, verbose= 1)

Train on 1610 samples, validate on 201 samples
Epoch 1/100
1610/1610 [==============================] - 1s 629us/step - loss: 0.0016 - acc: 0.0298 - val_loss: 9.2469e-04 - val_acc: 0.0100
Epoch 2/100
1610/1610 [==============================] - 1s 331us/step - loss: 0.0016 - acc: 0.0453 - val_loss: 9.2444e-04 - val_acc: 0.0100
Epoch 3/100
1610/1610 [==============================] - 1s 312us/step - loss: 0.0016 - acc: 0.0453 - val_loss: 9.2504e-04 - val_acc: 0.0100
Epoch 4/100
1610/1610 [==============================] - 1s 312us/step - loss: 0.0016 - acc: 0.0435 - val_loss: 9.2459e-04 - val_acc: 0.0100
Epoch 5/100
1610/1610 [==============================] - 0s 310us/step - loss: 0.0016 - acc: 0.0441 - val_loss: 9.2467e-04 - val_acc: 0.0100
Epoch 6/100
1610/1610 [==============================] - 0s 310us/step - loss: 0.0015 - acc: 0.0466 - val_loss: 9.2481e-04 - val_acc: 0.0149
Epoch 7/100
1610/1610 [==============================] - 1s 321us/step - loss: 0.0016 - acc: 0.0354 - val_l

1610/1610 [==============================] - 0s 308us/step - loss: 0.0016 - acc: 0.0447 - val_loss: 9.2639e-04 - val_acc: 0.0100
Epoch 59/100
1610/1610 [==============================] - 0s 310us/step - loss: 0.0016 - acc: 0.0447 - val_loss: 9.2610e-04 - val_acc: 0.0100
Epoch 60/100
1610/1610 [==============================] - 0s 310us/step - loss: 0.0016 - acc: 0.0447 - val_loss: 9.2557e-04 - val_acc: 0.0100
Epoch 61/100
1610/1610 [==============================] - 1s 313us/step - loss: 0.0016 - acc: 0.0453 - val_loss: 9.2512e-04 - val_acc: 0.0100
Epoch 62/100
1610/1610 [==============================] - 1s 320us/step - loss: 0.0016 - acc: 0.0429 - val_loss: 9.2501e-04 - val_acc: 0.0100
Epoch 63/100
1610/1610 [==============================] - 1s 312us/step - loss: 0.0016 - acc: 0.0447 - val_loss: 9.2513e-04 - val_acc: 0.0100
Epoch 64/100
1610/1610 [==============================] - 0s 310us/step - loss: 0.0016 - acc: 0.0447 - val_loss: 9.2524e-04 - val_acc: 0.0100
Epoch 65/100
1610/1

In [36]:
test_sgd = x_test.copy()
test_adam = x_test.copy()

In [37]:
error_sgd = []
for (index, element) in enumerate(y_test):
    predict = model_sgd.predict(test_sgd, verbose = 0)
    error_sgd.append(mean_square_error(element, predict[0]))
    test_sgd = findNewXTest(test_sgd, predict[0])

In [38]:
error_adam = []
for (index, element) in enumerate(y_test):
    predict = model_adam.predict(test_adam, verbose = 0)
    error_adam.append(mean_square_error(element, predict[0]))
    test_adam = findNewXTest(test_adam, predict[0])

In [40]:
for (index, element) in enumerate(y_test):
    print("round", index, ":", error_sgd[index], error_adam[index], error_sgd[index] - error_adam[index])
    
print(sum(error_sgd), sum(error_adam))

round 0 : 0.00066594174307 0.000667250877993 -1.30913492246e-06
round 1 : 0.000818180652858 0.0008213196626 -3.13900974266e-06
round 2 : 0.000705636720748 0.00071119942773 -5.56270698159e-06
round 3 : 0.000816184234298 0.000824189024086 -8.00478978801e-06
round 4 : 0.000697512968248 0.000700713118607 -3.20015035906e-06
round 5 : 0.000945144139829 0.000950547277623 -5.4031377937e-06
round 6 : 0.00100212401495 0.00100546167169 -3.33765673572e-06
round 7 : 0.000912119549368 0.000916068131558 -3.94858219002e-06
round 8 : 0.00128159532257 0.00127777195666 3.82336590764e-06
round 9 : 0.000917146880238 0.000914036093869 3.1107863684e-06
round 10 : 0.000772354172522 0.000773742445004 -1.38827248212e-06
round 11 : 0.00135696530187 0.00135594242655 1.02287531766e-06
round 12 : 0.00111079249442 0.00110912189663 1.6705977906e-06
round 13 : 0.00131877277543 0.00132263601658 -3.86324114927e-06
round 14 : 0.0010610469099 0.00106253963965 -1.49272975265e-06
round 15 : 0.000744956608903 0.0007462305723